# Deep Learning model to detect DGA domains

This notebook uses a pre-trained deep learning model to predict whether a domain is DGA generated or not.

### What are DGA domains ?

Adversaries may make use of Domain Generation Algorithms (DGAs) to dynamically identify a destination domain for command and control traffic rather than relying on a list of static IP addresses or domains. This has the advantage of making it much harder for defenders to block, track, or take over the command and control channel, as there potentially could be thousands of domains that malware can check for instructions.

DGAs can take the form of apparently random or "gibberish" strings (ex: istgmxdejdnxuyla.ru) when they construct domain names by generating each letter. Alternatively, some DGAs employ whole words as the unit by concatenating words together instead of letters (ex: cityjulydish.net). Many DGAs are time-based, generating a different domain for each time period (hourly, daily, monthly, etc). Others incorporate a seed value as well to make predicting future domains more difficult for defenders https://attack.mitre.org/techniques/T1568/002/

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [32]:
import time
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dropout,Dense,Activation
from tensorflow.keras.activations import sigmoid
import matplotlib.pyplot as plt
import math
import numpy as np
import pickle
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing import sequence
import datetime
from collections import Counter
MODEL_DIRECTORY = "/srv/app/model/data/"
MAX_VOCAB = 40
pd.options.mode.chained_assignment = None 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("tensorflow version: " + tf.__version__)

## Stage 1 - get a data sample from Splunk
This notebook does not train

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("pretrained_dga_model_dsdl")
print(df.describe())
print(param)

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = tf.keras.models.load_model(MODEL_DIRECTORY + "pretrained_dga_model_dsdl")
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
tokenizer = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_model_dsdl/tokenizer", 'rb'))
vectorizer_words = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_model_dsdl/vectorizer_words", 'rb'))
vectorizer_non_dga_domains = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_model_dsdl/vectorizer_non_dga_domains", 'rb'))
weight_words = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_model_dsdl/weight_words", 'rb'))
weight_non_dga_grams = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_model_dsdl/weight_non_dga_grams", 'rb'))
domains = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_model_dsdl/domains", 'rb'))
model = tf.keras.models.load_model(MODEL_DIRECTORY + "pretrained_dga_model_dsdl")

def entropy(domain):
        p, lns = Counter(domain), float(len(domain))
        return -sum( count/lns * math.log(count/lns, 2) for count in p.values())
    
alexa_domains = domains['domain']

    
def is_in_alexa1m(domain,domains):
    return ((domain in domains))

    
    
def add_features(df):
    print ("1. Done adding ngram features")
    X_1= weight_words * vectorizer_words.transform(df['domain']).T 
    X_2= weight_non_dga_grams * vectorizer_non_dga_domains.transform(df['domain']).T
    X_3 =   df['domain'].map(lambda x: entropy(x)) 
    print ("2. Done adding entropy")
    X_4 = df['domain'].map(lambda x: len(x))
    print ("3. Done adding length of domain")
    in_alexa = set(alexa_domains) & set(df['domain'])
    X_5 = df['domain'].map(lambda x: is_in_alexa1m(x, in_alexa))
    print ("4. Done adding domain present in alexa domains")
    X_5 = X_5.astype(int)
    input2 = np.c_[X_1,X_2,X_3,X_4,X_5] #
    print ("appending done")
    return input2

def prep_text(texts):
    text_sequences = tokenizer.texts_to_sequences(texts)
    return sequence.pad_sequences(text_sequences, maxlen=MAX_VOCAB)

def apply(model,df,param):
    input1 = prep_text(df['domain'])
    input2 = add_features(df)
    
    yhat = model.predict([input1,input2])
    y_ = (yhat > 0.5).astype("int32")
    output = pd.DataFrame()
    #output['domain']  = df['domain']
    #output['true_dga']  = df['is_dga'] 
    output['pred_dga']=y_.ravel()
    output['pred_dga_proba']=yhat.ravel()
    return output

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [3]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = tf.keras.models.load_model(MODEL_DIRECTORY + "pretrained_dga_model_dsdl")
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code